# LogicGate

### 외부 함수

#### external function (sigmoid, numerical_derivative)

In [21]:
import numpy as np

In [22]:
# 시그모이드 함수
def sigmoid(x):
    return 1/(1+np.exp(-x))



# 수치미분 함수
def numerical_derivative(f,x): # x에 변수가 여러개
    delta_x = 1e-4
    grad = np.zeros_like(x)
    # 주어진 어레이와 같은 형태와 타입을 갖는 0으로 채워진 어레이를 반환
    # grad는 계산된 수치미분 값을 저장할 변수
    it = np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    #  op_flags를 이용하면 기본값은 읽기 전용이지만 읽기/쓰기 또는 
    # 쓰기 전용 모드로 설정할 수 있다
    # multi_index는 뭐지..?
    while not it.finished:
        idx = it.multi_index
        
        tmp_val = x[idx]
        # x의 원본값이 변화되는 것을 막고자 임시변수 tmp_val 이라는 곳에
        # 원본값을 저장해두고 있다
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_X)
        
        x[idx]=tmp_val - delta_x
        fx2 = f(x) # f(x-delta_X)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
    
    return grad

#### LogicGate class

In [23]:
class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)
        self.__tdata = tdata.reshape(4,1)
        
        # 가중치 W, 바이어스 b 초기화
        self.__W = np.random.rand(2,1)
        self.__b = np.random.rand(1)
        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
        
    # 손실함수
    def __loss_func(self):
        delta = 1e-7
        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)
        
        return -np.sum(self.__tdata*np.log(y+delta)+(1-self.__tdata)*np.log((1-y)+delta))
    
    # 손실 값 계산
    def error_val(self):
        delta = 1e-7
        
        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)
        
        return -np.sum(self.__tdata*np.log(y+delta)+(1-self.__tdata)*np.log((1-y)+delta))
    
    def train(self):
        f = lambda x : self.__loss_func()
        print("Initial erorr value = ", self.error_val())

        for step in range(8001):

            self.__W -= self.__learning_rate*numerical_derivative(f,self.__W)
            self.__b -= self.__learning_rate*numerical_derivative(f,self.__b)
            if(step % 400 == 0):
                print("step = ", step )
                print("error_value = ", self.error_val())
                    
    # 학습을 모두 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
    def predict(self, input_data):

        z=np.dot(input_data,self.__W)+self.__b
        y=sigmoid(z)

        if y>0.5:
            result = 1
        else:
            result = 0
        return y, result

### AND GATE 검증

In [24]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([0,0,0,1]) # 정답데이터

AND_obj = LogicGate("AND_GATE", xdata, tdata)
AND_obj.train()

Initial erorr value =  4.983359130665479
step =  0
error_value =  4.920477327061232
step =  400
error_value =  1.568791971821438
step =  800
error_value =  1.1579619414990994
step =  1200
error_value =  0.9280894658013815
step =  1600
error_value =  0.777373069321113
step =  2000
error_value =  0.6694488899843059
step =  2400
error_value =  0.5878154027711444
step =  2800
error_value =  0.5237120762738244
step =  3200
error_value =  0.47197218255526097
step =  3600
error_value =  0.42931530996592715
step =  4000
error_value =  0.39354292980400923
step =  4400
error_value =  0.3631194694076938
step =  4800
error_value =  0.33693683652448925
step =  5200
error_value =  0.3141739325301419
step =  5600
error_value =  0.2942087528122908
step =  6000
error_value =  0.2765612152350049
step =  6400
error_value =  0.2608547575328354
step =  6800
error_value =  0.24678983057940157
step =  7200
error_value =  0.2341251709106549
step =  7600
error_value =  0.22266429876961358
step =  8000
error_va

In [25]:
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])
for input_data in test_data:
    (sigmoid_val, logical_val) = AND_obj.predict(input_data)
    print(input_data, "=", logical_val, "\n")

[0 0] = 0 

[0 1] = 0 

[1 0] = 0 

[1 1] = 1 



### OR GATE, NAND GATE 모두 검증가능
### ==> XOR GATE의 경우 손실함수 값이 줄지않음

In [26]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([1,1,1,0]) # 정답데이터

NAND_obj = LogicGate("NAND_GATE", xdata, tdata)
NAND_obj.train()

test_data = np.array([[0,0],[0,1],[1,0],[1,1]])
for input_data in test_data:
    (sigmoid_val, logical_val) = NAND_obj.predict(input_data)
    print(input_data, "=", logical_val, "\n")

print("---------------------------------------")

xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([0,1,1,1]) # 정답데이터

OR_obj = LogicGate("OR_GATE", xdata, tdata)
OR_obj.train()

test_data = np.array([[0,0],[0,1],[1,0],[1,1]])
for input_data in test_data:
    (sigmoid_val, logical_val) = OR_obj.predict(input_data)
    print(input_data, "=", logical_val, "\n")

Initial erorr value =  2.638056641233497
step =  0
error_value =  2.6337874013316673
step =  400
error_value =  1.6003815292580978
step =  800
error_value =  1.1743587641969406
step =  1200
error_value =  0.9383401171917416
step =  1600
error_value =  0.7844837481294966
step =  2000
error_value =  0.6747090938226588
step =  2400
error_value =  0.5918791284657003
step =  2800
error_value =  0.5269512645623807
step =  3200
error_value =  0.47461632753142174
step =  3600
error_value =  0.4315147827299829
step =  4000
error_value =  0.3954008725783206
step =  4400
error_value =  0.36470921857403726
step =  4800
error_value =  0.33831207022445353
step =  5200
error_value =  0.31537489254745843
step =  5600
error_value =  0.2952662227083737
step =  6000
error_value =  0.27749915377092066
step =  6400
error_value =  0.2616920936344728
step =  6800
error_value =  0.24754171771853278
step =  7200
error_value =  0.23480388283038325
step =  7600
error_value =  0.22327988128948528
step =  8000
err

#### 해결 방법: NAND, OR, AND 조합으로 XOR 구현

In [27]:
input_data = np.array([[0,0],[0,1],[1,0],[1,1]])

s1 = [] #NAND 출력
s2 = [] #OR 출력

new_input_data = [] #AND 입력
final_output = []   #AND 출력

for index in range(len(input_data)):
    s1 = NAND_obj.predict(input_data[index])
    s2 = OR_obj.predict(input_data[index])
    
    new_input_data.append(s1[-1])
    new_input_data.append(s2[-1])
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)
    new_input_data = []
    
for index in range(len(input_data)):
    print(input_data[index]," = ", final_output[index], end='')
    print('\n')



[0 0]  =  0

[0 1]  =  1

[1 0]  =  1

[1 1]  =  0



#### => 머신러닝 XOR 문제는 다양한 GATE의 조합인 Multi-Layer로 해결할 수 있음
#### 이전 Gate의 출력이 다음 Gate의 입력으로 들어간다는 아이디어는 신경망 기반의 딥러닝의 핵심 아이디어이다.